In [39]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import re
import time
import string
from nltk.corpus import stopwords
import pandas as pd  
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis.gensim_models
import os
from nltk.tokenize import RegexpTokenizer,sent_tokenize,word_tokenize
import pandas as pd
import numpy as np
import pprint
from gensim.models import CoherenceModel
import random
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score,accuracy_score

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
input_dir = "E:/UOR Notes/Stats for Machine Learning/Assignments/Kaggle Assignment/"
training_input_file = "Processed_Training_Data_comb_1.csv"
testing_input_file = "Processed_Testing_Data_Comb1.csv"
training_input_path = input_dir+training_input_file
testing_input_path = input_dir + testing_input_file
random.seed(265)  #Setting random seed

In [6]:
cong_tweets_data = pd.read_csv(training_input_path)
cong_tweets_data.head()

,Unnamed: 0,favorite_count,full_text,hashtags,retweet_count,year,party_id,Cleaned_Text_WO_Stopwords
0,0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,kusinews one longtime viewer congressman darre...
1,1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,urging cdcgov immediately launch phone hotline...
2,2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,tomorrow #mo03 senior graduate calvary luthera...
3,3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,congrats #teamusa canton native jgreenway12 wi...
4,4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,pleased amergateways june fiesta honored #immi...


In [7]:
cong_testing_data = pd.read_csv(testing_input_path)
cong_testing_data.head()

,Unnamed: 0,Id,favorite_count,full_text,hashtags,retweet_count,year,party,Cleaned_Text_WO_Stopwords
0,0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D,#taxreform improved playing field worker busin...
1,1,1,27,"b'This #NativeWomensEqualPay Day, we recommit ...",NativeWomensEqualPay,11,NaN,D,#nativewomensequalpay recommit passing paychec...
2,2,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D,became convinced generation silence make compl...
3,3,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D,#nationaladoptionmonth honor adoptive parent p...
4,4,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D,happy #airborneday usarmy paratrooper veteran ...


In [11]:
cong_tweets_data['Cleaned_Text_WO_Stopwords'].dropna(inplace=True)
docs = np.array(cong_tweets_data['Cleaned_Text_WO_Stopwords'])
docs = [doc for doc in docs if str(doc) != 'nan']

In [12]:
def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    print(len(docs))
    for idx in range(len(docs)):
        #print(idx)
        #print(docs[idx])
                #docs[idx] = [[word.lower() for word in docs[idx].split()] for line in data] # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.
        #print(docs[idx])
            # Remove numbers, but not words that contain numbers.
    docs_token = [[token.lower() for token in doc if not token.isdigit()] for doc in docs]

            # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]

            # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

    return docs

In [13]:
docs = docs_preprocessor(docs)

592803


In [14]:
bigram = Phrases(docs, min_count=10)
bigram

In [15]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique words in initital documents: 260893
Number of unique words after removing rare and common words: 29198
Number of unique tokens: 29198
Number of documents: 592803


In [16]:
import time
# Set training parameters.
num_topics = range(5,6)
chunksize = 100 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

topic_hp_df = pd.DataFrame(columns = ["Event Name","Number of Topics","Coherence Score","Perplexity Score","Iterations","Chunksize","Time Taken","Keywords in Topics"])


for num in num_topics:
    print(num)
    inter_topic_df = pd.DataFrame(columns = ["Event Name","Number of Topics","Coherence Score","Perplexity Score","Iterations","Chunksize","Time Taken","Keywords in Topics"])
    # Make a index to word dictionary.
    if len(dictionary)>0:
        temp = dictionary[0]  # This is only to "load" the dictionary.
        start = time.time()
        id2word = dictionary.id2token
        model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize,alpha='auto', eta='auto', iterations=iterations, num_topics=num,passes=passes, eval_every=eval_every)
        end = time.time()
        print(end-start)
        time_taken = end-start

        Y = model.print_topics(num_words = 10)
        topic_words_list = []
        for i in range(0,len(Y)):
            #topic_words_list.append(html.Br())
            topic_words_list.append("Topic "+str(i+1)+": ")
            X = Y[i][1]
            a = X.split("+")
            words_str = ""
            for j in range(0,len(a)):
                p = a[j].split("*")[1].replace("\"","")
                words_str += p
            topic_words_list.append(words_str)

        perplexity = model.log_perplexity(corpus)
        coherence_model_lda = CoherenceModel(model=model, texts=docs, dictionary=dictionary, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        
        
        inter_topic_df["Number of Topics"] = [num]
        inter_topic_df["Coherence Score"] = [coherence_lda]
        inter_topic_df["Perplexity Score"] = [perplexity]
        inter_topic_df["Iterations"] = [iterations]
        inter_topic_df["Chunksize"] = [chunksize]
        inter_topic_df["Time Taken"] = [time_taken]
        inter_topic_df["Keywords in Topics"] = [topic_words_list]
        topic_hp_df = pd.concat([topic_hp_df,inter_topic_df])
        


#topic_hp_df.to_csv(input_dir+"Topic_Modelling_Hyperparameter_Tuning.csv")
#topic_hp_df

5
13337.203060626984


NameError: name 'event_name' is not defined

In [24]:
cong_tweets_data.columns

Index(['Unnamed: 0', 'favorite_count', 'full_text', 'hashtags',
       'retweet_count', 'year', 'party_id', 'Cleaned_Text_WO_Stopwords'],
      dtype='object')

In [28]:
tweet_train = cong_tweets_data
train_vecs = []
for i in range(tweet_train.shape[0]):
    #print(i)
    top_topics = (
        model.get_document_topics(corpus[i],
                                      minimum_probability=0.0)
    )
    #print(top_topics)
    topic_vec = [top_topics[j][1] for j in range(5)]
    topic_vec.extend([tweet_train.iloc[i].favorite_count])
    topic_vec.extend([tweet_train.iloc[i].retweet_count])
    topic_vec.extend([len(tweet_train["full_text"].iloc[i])])
    train_vecs.append(topic_vec)

In [31]:
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
cong_tweets_data['encoded_party']= label_encoder.fit_transform(cong_tweets_data["party_id"])

In [43]:
X = np.array(train_vecs)
y = np.array(cong_tweets_data.encoded_party)

kf = KFold(5, shuffle=True, random_state=42)
cv_lr_acc, cv_lrsgd_acc, cv_svcsgd_acc,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='liblinear',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_acc.append(accuracy_score(y_val, y_pred))
    
    # Logistic Regression SGD
    sgd = SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_acc.append(accuracy_score(y_val, y_pred))
    
    # SGD Modified Huber
    sgd_huber = SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_acc.append(accuracy_score(y_val, y_pred))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_acc):.3f} +- {np.std(cv_svcsgd_acc):.3f}')


Logistic Regression Val f1: 0.600 +- 0.001
Logisitic Regression SGD Val f1: 0.602 +- 0.005
SVM Huber Val f1: 0.510 +- 0.046


In [44]:
print(f'Logistic Regression Val f1: {np.mean(cv_lr_acc):.3f} +- {np.std(cv_lr_acc):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_acc):.3f} +- {np.std(cv_lrsgd_acc):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_acc):.3f} +- {np.std(cv_svcsgd_acc):.3f}')

Logistic Regression Val f1: 0.599 +- 0.001
Logisitic Regression SGD Val f1: 0.600 +- 0.003
SVM Huber Val f1: 0.510 +- 0.046


### Topic Modelling on Test Data

In [45]:
cong_testing_data['Cleaned_Text_WO_Stopwords'].dropna(inplace=True)
test_docs = np.array(cong_testing_data['Cleaned_Text_WO_Stopwords'])
test_docs = [doc for doc in test_docs if str(doc) != 'nan']

In [46]:
test_docs = docs_preprocessor(test_docs)

265000


In [47]:
test_corpus = [train_id2word.doc2bow(text) for text in test_docs]

test_vecs = []
for i in range(len(cong_testing_data.shape[0])):
    top_topics = model.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(5)]
    topic_vec.extend([rev_test.iloc[i].real_counts]) # counts of reviews for restaurant
    topic_vec.extend([len(rev_test.iloc[i].text)]) # length review
    test_vecs.append(topic_vec)

Number of unique words in initital documents: 159400
Number of unique words after removing rare and common words: 17966
Number of unique tokens: 17966
Number of documents: 592803


In [48]:
import time
# Set training parameters.
num_topics = range(5,6)
chunksize = 100 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

topic_hp_df = pd.DataFrame(columns = ["Event Name","Number of Topics","Coherence Score","Perplexity Score","Iterations","Chunksize","Time Taken","Keywords in Topics"])


for num in num_topics:
    print(num)
    inter_topic_df = pd.DataFrame(columns = ["Event Name","Number of Topics","Coherence Score","Perplexity Score","Iterations","Chunksize","Time Taken","Keywords in Topics"])
    # Make a index to word dictionary.
    if len(test_dictionary)>0:
        temp = test_dictionary[0]  # This is only to "load" the dictionary.
        start = time.time()
        test_id2word = test_dictionary.id2token
        test_model = LdaModel(corpus=test_corpus, id2word=test_id2word, chunksize=chunksize,alpha='auto', eta='auto', iterations=iterations, num_topics=num,passes=passes, eval_every=eval_every)
        end = time.time()
        print(end-start)
        time_taken = end-start

        Y = test_model.print_topics(num_words = 10)
        topic_words_list = []
        for i in range(0,len(Y)):
            #topic_words_list.append(html.Br())
            topic_words_list.append("Topic "+str(i+1)+": ")
            X = Y[i][1]
            a = X.split("+")
            words_str = ""
            for j in range(0,len(a)):
                p = a[j].split("*")[1].replace("\"","")
                words_str += p
            topic_words_list.append(words_str)

        perplexity = model.log_perplexity(corpus)
        coherence_model_lda = CoherenceModel(model=model, texts=docs, dictionary=dictionary, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        
        
        inter_topic_df["Number of Topics"] = [num]
        inter_topic_df["Coherence Score"] = [coherence_lda]
        inter_topic_df["Perplexity Score"] = [perplexity]
        inter_topic_df["Iterations"] = [iterations]
        inter_topic_df["Chunksize"] = [chunksize]
        inter_topic_df["Time Taken"] = [time_taken]
        inter_topic_df["Keywords in Topics"] = [topic_words_list]
        topic_hp_df = pd.concat([topic_hp_df,inter_topic_df])
        


5


KeyboardInterrupt: 